<a href="https://colab.research.google.com/github/colinmcnamara/Digital_Deming/blob/main/Digital_Deming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up your local environment 

On intial run of "import colab_env" it will create vars.env in the root of your Google Drive

COLAB_ENV = Active

export OPENAI_API_KEY = "your_key"

export SERPAPI_API_KEY = "your_key"


#Backend Code

Install my local packages

In [1]:
pip install OpenAI langchain pydantic faiss-gpu colab-env unstructured chromadb tiktoken pdf2image faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 81.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 75.5 

Yo, let's import our local env so we don't have to show the world our keys

In [2]:
import colab_env

Mounted at /content/gdrive


Let's import our python modules

In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import OpenAI, VectorDBQA
from langchain.chains import RetrievalQA
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.document_loaders import DirectoryLoader
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
import magic
import os
import nltk
import textwrap



Let's set the temperature of our AI models

In [4]:
llm = OpenAI(temperature=0)
chat = ChatOpenAI(temperature=0)

Let's specify the directory and file type for our Demming source docs

In [5]:
loader = DirectoryLoader('/content/gdrive/MyDrive/Projects/Digital_Deming/src_content', glob='**/*.pdf')

Load our documents


In [6]:
documents = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Check how many documents are loaded

In [7]:
len (documents)

1

Let's split our document up into chunks

In [8]:
#text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap  = 20,
    length_function = len,
    separators=["\n\n", "\n", " ", ""]
)

Let' create a variable named texts, and split these documents into it

In [9]:
texts = text_splitter.split_documents(documents)

Let's tell it that we want it to use OpenAI embeddings, and tell it what env variable the key is held

In [10]:
embeddings = OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY'])

Let's create a variable named docsearch, and pull from Chroma vectorstore including texts, and the embeddings)

In [11]:
docsearch = Chroma.from_documents(texts, embeddings)

define qa chain, which pulls from VectorDBQA, and use the LLM chain named OpenAI. We specify that it "Stuff"'s all the data in. We could use "map_reduce" instead to optimize for performance and scale. Let's tell it to use the vectorstore we defined earlier named docsearch

note, i'm trying RetrievalQA because VectorDBQA is depricated

In [12]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch, return_source_documents=True)
query = "Describe in depth, what is the foundation of quality. Provide three examples using bullet points"
result = qa({"query": query})

/usr/local/lib/python3.10/dist-packages/langchain/chains/retrieval_qa/base.py:206: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


In [13]:
result['result']

' \nThe foundation of quality is a set of principles and processes that prioritize customer satisfaction, quality, and continual improvement. \n\nExamples: \n• Customer satisfaction: organizations should strive to meet or exceed customer expectations. \n• Quality: organizations should aim to produce products and services that meet or exceed the highest standards of quality. \n• Continual improvement: organizations should continually seek to improve their products, services, and processes.'

In [ ]:
result['source_documents']

[Document(page_content='1) AGREE: Advisory Group on Reliability of Electronic Equipment 2) AOTS: Association for Overseas Technical Scholarship 3) APO: Asian Productivity Organization 4) ASA: American Standards Association 5) ASQC: American Society for Quality Control 6) BS: British Standards 7) CCS: Civil Communication Section 8) COSCO: Control System Committee 9) EOQC: European Organization for Quality Control 10) EOQ: European Organization for Quality 11) EFQM: European Foundation for Quality Management 12) FMES: Japan Federation of Managerial Engineering Societies\n\n524\n\nA Parallel Chronology of Prof. Kaoru Ishikawa’s Life and TQC in Japan', metadata={'source': '/content/gdrive/MyDrive/Projects/Digital_Deming/src_content/ParallelChronology_Ver2_150717.pdf'}),
 Document(page_content='Major Events in the Development of QC\n\nCourse. (July–Dec. 1957, JUSE)\n\n\uf09f The Chugoku Quality Control Research Group was established. (JSA Hiroshima) \uf09f The Design of Experiment Seminar w

In [17]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch, return_source_documents=True)
#query = "what is the impact of JUSE on TQM?"
#query = "what is the purpouse of life?"
#query = "what does demming like for lunch"
#query = "is quality like a human"
#query = "what is the fourth rule of TQM"
#query = "what is the most important concept in TQM"
#query =  "WHo won the deming prize?"
#query = "Is TQM like lean?"
#query = "How are TQM and Lean different?"
query = "What is lean?"

result = qa({"query": query})

In [18]:
result['result']

" I don't know."

In [ ]:
result['source_documents']

[Document(page_content='Control and Standardization” was broadcast on NHK Educational TV: The Factories of the Future (59), Factory Management Series (60). QC Series (60), Standardization Series (60), and Standardization for Small Enterprises (61).\n\n\uf09f He became a professor of the Faculty of Engineering, the University of Tokyo.\n\n\uf09f He became the chair of the Quality\n\nMonth Committee. (–89)\n\n\uf09f The TV program The Factories of the Future was broadcast. (–61, NHK Educational TV)\n\n\uf09f The Seminar on How to Introduce QC\n\n\uf09f He became the chief of the Standards\n\nwas held. (JSA)\n\nRationalization Research Subgroup of the COSCO Sampling Inspection Research Group. (–72, JSA)\n\n\uf09f He proposed “Ten QC Principles for\n\nVendee-Vendor Relations.”\n\n\uf09f He appeared as a lecturer on the TV\n\neducational program series The Factories of the Future, (–March 1961, NHK Educational TV) \uf09f He published Quality Control Textbook for Foremen Volume A and Volume 

# Playground

In [19]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch, return_source_documents=True)
#query = "what is the impact of JUSE on TQM?"
#query = "what is the purpouse of life?"
#query = "what does demming like for lunch"
#query = "is quality like a human"
#query = "what is the fourth rule of TQM"
query = "what is Total Quality Management (TQM)"
result = qa({"query": query})

In [20]:
result['result']

' Total Quality Management (TQM) is a management philosophy that focuses on the continuous improvement of processes and services in order to ensure customer satisfaction. It is a business approach that seeks to eliminate waste and improve efficiency and effectiveness.'

In [ ]:
result['source_documents']

[Document(page_content='1) AGREE: Advisory Group on Reliability of Electronic Equipment 2) AOTS: Association for Overseas Technical Scholarship 3) APO: Asian Productivity Organization 4) ASA: American Standards Association 5) ASQC: American Society for Quality Control 6) BS: British Standards 7) CCS: Civil Communication Section 8) COSCO: Control System Committee 9) EOQC: European Organization for Quality Control 10) EOQ: European Organization for Quality 11) EFQM: European Foundation for Quality Management 12) FMES: Japan Federation of Managerial Engineering Societies\n\n524\n\nA Parallel Chronology of Prof. Kaoru Ishikawa’s Life and TQC in Japan', metadata={'source': '/content/gdrive/MyDrive/Projects/Digital_Deming/src_content/ParallelChronology_Ver2_150717.pdf'}),
 Document(page_content='Medal. (ASQC)\n\n\uf09f The Pharmaceutical Manufacturing and Quality Control (GMP) Seminar was held. (Sep., JUSE)\n\n\uf09f The Japan Prefabricated Construction\n\nSuppliers and Manufacturers Associa

In [23]:
# Create a list to stuff section results into 
introduction = []

#append result  to the array
introduction.append(result)

#print the array
print(introduction)

[{'query': 'what is Total Quality Management (TQM)', 'result': ' Total Quality Management (TQM) is a management philosophy that focuses on the continuous improvement of processes and services in order to ensure customer satisfaction. It is a business approach that seeks to eliminate waste and improve efficiency and effectiveness.', 'source_documents': [Document(page_content='1) AGREE: Advisory Group on Reliability of Electronic Equipment 2) AOTS: Association for Overseas Technical Scholarship 3) APO: Asian Productivity Organization 4) ASA: American Standards Association 5) ASQC: American Society for Quality Control 6) BS: British Standards 7) CCS: Civil Communication Section 8) COSCO: Control System Committee 9) EOQC: European Organization for Quality Control 10) EOQ: European Organization for Quality 11) EFQM: European Foundation for Quality Management 12) FMES: Japan Federation of Managerial Engineering Societies\n\n524\n\nA Parallel Chronology of Prof. Kaoru Ishikawa’s Life and TQC 

In [ ]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch, return_source_documents=True)
#query = "what is the impact of JUSE on TQM?"
#query = "what is the purpouse of life?"
#query = "what does demming like for lunch"
#query = "is quality like a human"
#query = "what is the fourth rule of TQM"
query = "Write a detailed introduction of the significance of TQM in the modern business world"
result = qa({"query": query})

In [ ]:
result['result']

' Total Quality Management (TQM) is a business approach that aims to improve the quality of products and services through the implementation of rigorous processes and procedures. TQM strives to create a customer-centric environment by identifying and addressing customer needs and expectations. It also emphasizes the importance of continual improvement and encourages the involvement of all employees in the process. TQM is particularly important in the modern business world as it helps to create a competitive advantage, reduce costs, increase efficiency and productivity, and improve customer satisfaction. By implementing TQM, organizations can ensure that their products and services meet the highest quality standards and are able to remain competitive in the market.'

In [ ]:
#append result  to the array
introduction.append(result)

#print the array
print(introduction)

[{'query': 'what is Total Quality Management (TQM)', 'result': ' Total Quality Management (TQM) is an approach to improving the quality of products and services by focusing on customer satisfaction and continual improvement of all organizational processes. TQM focuses on eliminating defects and waste, setting and meeting customer requirements, establishing performance standards, and measuring and reporting performance results.', 'source_documents': [Document(page_content='1) AGREE: Advisory Group on Reliability of Electronic Equipment 2) AOTS: Association for Overseas Technical Scholarship 3) APO: Asian Productivity Organization 4) ASA: American Standards Association 5) ASQC: American Society for Quality Control 6) BS: British Standards 7) CCS: Civil Communication Section 8) COSCO: Control System Committee 9) EOQC: European Organization for Quality Control 10) EOQ: European Organization for Quality 11) EFQM: European Foundation for Quality Management 12) FMES: Japan Federation of Manag

In [ ]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch, return_source_documents=True)
#query = "what is the impact of JUSE on TQM?"
#query = "what is the purpouse of life?"
#query = "what does demming like for lunch"
#query = "is quality like a human"
#query = "what is the fourth rule of TQM"
#query = "Write a detailed introduction of the significance of TQM in the modern business world"
query = "Introduction to Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)"
result = qa({"query": query})

In [ ]:
result['result']

' In 1962, Kaoru Ishikawa was a lecturer of the Introductory Course for Design of Experiment at the University of Tokyo. He was also a member of the ISO/TC 102 Japan Committee and served as the chair of the Sampling Subcommittee. He became a lecturer of the QC Seminar for Managers at the Central Japan Industries Association, and he became the chair of the editorial board of Genba To QC-magazine at JUSE. He was also the chair of the Editorial Committee of the JUSE English Report and the chair of the Organizing Committee of the JUSE Course for Purchasing.'

In [ ]:
#append result  to the array
introduction.append(result)

#print the array
print(introduction)

[{'query': 'what is Total Quality Management (TQM)', 'result': ' Total Quality Management (TQM) is an approach to improving the quality of products and services by focusing on customer satisfaction and continual improvement of all organizational processes. TQM focuses on eliminating defects and waste, setting and meeting customer requirements, establishing performance standards, and measuring and reporting performance results.', 'source_documents': [Document(page_content='1) AGREE: Advisory Group on Reliability of Electronic Equipment 2) AOTS: Association for Overseas Technical Scholarship 3) APO: Asian Productivity Organization 4) ASA: American Standards Association 5) ASQC: American Society for Quality Control 6) BS: British Standards 7) CCS: Civil Communication Section 8) COSCO: Control System Committee 9) EOQC: European Organization for Quality Control 10) EOQ: European Organization for Quality 11) EFQM: European Foundation for Quality Management 12) FMES: Japan Federation of Manag

In [ ]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch, return_source_documents=True)
#query = "what is the impact of JUSE on TQM?"
#query = "what is the purpouse of life?"
#query = "what does demming like for lunch"
#query = "is quality like a human"
#query = "what is the fourth rule of TQM"
#query = "Write a detailed introduction of the significance of TQM in the modern business world"
#query = "Introduction to Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)"
query = " Write a Thesis Statement regarding TQM, Kaoru Ishikawa, Japanese Union of Scientists and engineers."
result = qa({"query": query})

In [ ]:
result['result']

" Thesis Statement: Kaoru Ishikawa's contributions to Total Quality Management, through his research and involvement in the Japanese Union of Scientists and Engineers, have been instrumental in the development of quality control in Japan."

In [ ]:
#append result  to the array
introduction.append(result)

#print the array
print(introduction)

[{'query': 'what is Total Quality Management (TQM)', 'result': ' Total Quality Management (TQM) is an approach to improving the quality of products and services by focusing on customer satisfaction and continual improvement of all organizational processes. TQM focuses on eliminating defects and waste, setting and meeting customer requirements, establishing performance standards, and measuring and reporting performance results.', 'source_documents': [Document(page_content='1) AGREE: Advisory Group on Reliability of Electronic Equipment 2) AOTS: Association for Overseas Technical Scholarship 3) APO: Asian Productivity Organization 4) ASA: American Standards Association 5) ASQC: American Society for Quality Control 6) BS: British Standards 7) CCS: Civil Communication Section 8) COSCO: Control System Committee 9) EOQC: European Organization for Quality Control 10) EOQ: European Organization for Quality 11) EFQM: European Foundation for Quality Management 12) FMES: Japan Federation of Manag

Let's see if we can output this to markdown

In [ ]:
concatenated_data = [str(item) for item in introduction]
for entry in concatenated_data:
    print("Processing entry: ")
    messages = [
        SystemMessage(content="You are an AI that takes dictionary data and translates it into well formatted markdown. You create sections and subsections to communicate as much information as possible"),
        HumanMessage(content=entry)
    ]
    result = chat(messages)

    print("Result: ", result)

Processing entry: 


Result:  content='# Total Quality Management (TQM)\n\nTotal Quality Management (TQM) is an approach to improving the quality of products and services by focusing on customer satisfaction and continual improvement of all organizational processes. TQM focuses on eliminating defects and waste, setting and meeting customer requirements, establishing performance standards, and measuring and reporting performance results.\n\n## Source Documents\n\n- AGREE: Advisory Group on Reliability of Electronic Equipment\n- AOTS: Association for Overseas Technical Scholarship\n- APO: Asian Productivity Organization\n- ASA: American Standards Association\n- ASQC: American Society for Quality Control\n- BS: British Standards\n- CCS: Civil Communication Section\n- COSCO: Control System Committee\n- EOQC: European Organization for Quality Control\n- EOQ: European Organization for Quality\n- EFQM: European Foundation for Quality Management\n- FMES: Japan Federation of Managerial Engineering Societies\n\nThe 

In [137]:

  messages = [
      SystemMessage(content="You are an assistant helping me research a paper"),
      HumanMessage(content="return an outline for the book - The History and Evolution of Total Quality Management (TQM): The Contributions of Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE) ")
  ]
  result = chat(messages)

  print("Result: ", result)

Result:  content="I. Introduction\n- Brief overview of Total Quality Management (TQM)\n- Importance of studying the history and evolution of TQM\n- Thesis statement\n\nII. The Origins of TQM\n- Early quality control practices in manufacturing\n- The role of W. Edwards Deming in the development of TQM\n- The impact of World War II on TQM\n\nIII. Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)\n- Ishikawa's background and contributions to TQM\n- The formation and goals of JUSE\n- The development of Ishikawa's quality control tools\n\nIV. The Spread of TQM\n- The influence of JUSE on TQM in Japan\n- The adoption of TQM by American companies\n- The impact of TQM on global business practices\n\nV. Criticisms and Challenges of TQM\n- Criticisms of TQM's effectiveness\n- Challenges in implementing TQM in organizations\n- The evolution of TQM in response to criticisms and challenges\n\nVI. Conclusion\n- Summary of key points\n- The ongoing relevance of TQM in modern bu

In [136]:
#from langchain.prompts import PromptTemplate
#prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
#Provide responses in Markdown using # to indicate a heading and ## to indicate a subheading#

#{context}

#Question: {question}
#Answer as if you are a quality management consultant. Use specific dates and references. Use bullet points and tabels where appropriate. Reference source documents:

#"""
#PROMPT = PromptTemplate(
#    template=prompt_template, input_variables=["context", "question"]
#)
#chain_type_kwargs = {"prompt": PROMPT}
#qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=docsearch.as_retriever(), chain_type_kwargs=chain_type_kwargs)
query = "Who won the Deming prize?"
qa.run(query)

"# Deming Prize\nThe Deming Prize is awarded to organizations in recognition of their excellence in quality control and total quality management.\n\n## Winners of the Deming Prize\nThe first winners of the Deming Prize were Showa Denko, Tanabe Pharmaceuticals, Fuji Iron & Steel, and Yawata Iron & Steel. These organizations were recognized at the 1st Deming Prize Ceremony, which was held in Osaka on September 1952, by the Japanese Union of Scientists and Engineers (JUSE). \n\nMitsubishi Heavy Industries' Kobe Shipyard was the first winner of the Deming Application Prize, which was established in November 1980."

In [138]:
query = "Who won the Deming prize?"
qa.run(query)

'The first winners of the Deming Prize were: \n\n* Deming Prize for Individuals: Motosaburo Masuyama \n* Deming Application Prize: Showa Denko, Tanabe Pharmaceuticals, Fuji Iron & Steel, and Yawata Iron & Steel. \n\nThese awards were presented at the 1st Deming Prize Ceremony, which was held on September 1952 at the 1st QC Conference in Osaka, Japan and was organized by JUSE (Japan Union of Scientists and Engineers).'

In [ ]:
query = "Who won the Deming prize?"
qa.run(query)

'The first winners of the Deming Prize in 1952 were: \n\n• Deming Prize for Individuals: Motosaburo Masuyama \n• Deming Application Prize: Showa Denko, Tanabe Pharmaceuticals, Fuji Iron & Steel, and Yawata Iron & Steel. \n\nThe 1st Deming Prize Ceremony was held on September, 1952 at the 1st QC Conference in Osaka, Japan (JUSE).'

In [ ]:
query = "Who won the Deming prize?"
qa.run(query)

'The first winners of the Deming Prize in 1952 were: \n\n• Deming Prize for Individuals: Motosaburo Masuyama \n• Deming Application Prize: Showa Denko, Tanabe Pharmaceuticals, Fuji Iron & Steel, and Yawata Iron & Steel. \n\nThe 1st Deming Prize Ceremony was held on September, 1952 at the 1st QC Conference in Osaka, Japan (JUSE).'

In [ ]:
query = "Who won the Deming prize?"
qa.run(query)

'The first winners of the Deming Prize in 1952 were: \n\n• Deming Prize for Individuals: Motosaburo Masuyama \n• Deming Application Prize: Showa Denko, Tanabe Pharmaceuticals, Fuji Iron & Steel, and Yawata Iron & Steel. \n\nThe 1st Deming Prize Ceremony was held on September, 1952 at the 1st QC Conference in Osaka, Japan (JUSE).'

In [135]:
query = "Who won the Deming prize?"
qa.run(query)

'The first winners of the Deming Prize in 1952 were: \n\n• Deming Prize for Individuals: Motosaburo Masuyama \n• Deming Application Prize: Showa Denko, Tanabe Pharmaceuticals, Fuji Iron & Steel, and Yawata Iron & Steel. \n\nThe 1st Deming Prize Ceremony was held on September, 1952 at the 1st QC Conference in Osaka, Japan (JUSE).'

In [131]:
query = "when was the last deming prize awarded"
qa.run(query)

'# Answer\nThe last Deming Prize was awarded in November 1986. \n\nReferences:\n\nThe Deming Prize for Individuals was awarded to Motosaburo Masuyama in 1952. \n\nThe Deming Application Prize was awarded to Showa Denko, Tanabe Pharmaceuticals, Fuji Iron & Steel, and Yawata Iron & Steel in 1952. \n\nThe 15th Anniversary Party of the Deming Prize was held in November 1986.'

In [132]:
query = "who invented QC circle?"
qa.run(query)

'As a quality management consultant, I can say that the concept of QC circles originated in Japan in the early 1950s. \n\nThe invention of QC circles is attributed to Professor Kaoru Ishikawa, who was a professor at the University of Tokyo. \n\nThe concept was further developed by the Japanese Union of Scientists and Engineers (JUSE) in the late 1950s and early 1960s. \n\nIn 1961, JUSE launched the QC Circle Movement in Japan, which was based on the principles of Total Quality Control. \n\nThe first QC circles were established at Lockheed in the United States in the early 1960s. \n\nSanuier-Dubal, a French electric motor manufacturer, was the first to form QC circles in Europe in the mid-1960s.'

In [16]:
query = "when was the QC Circle Seminar for Top Management held"
qa.run(query)

ValueError: ignored

# The History and Evolution of Total Quality Management (TQM): The Contributions of Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)

In [ ]:
# Create a varible to stuff section results into 
book = []
result = []
#print the variable
print(book)

[]


## I. Introduction
* Definition of Total Quality Management (TQM)
* Brief introduction of the significance of TQM in the modern business world
* Introduction to Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)
* Thesis Statement

In [15]:
section = "#Definition of Total Quality Management (TQM)"
query = "write a definition of Total Quality Management (TQM)"
result = []
book = []
result = qa.run(query)

#append result  to the variable
#book.append(section)
book.append(result)

#print the variable
print(book)

ValueError: ignored

In [148]:
section = "#Brief introduction of the significance of TQM in the modern business world)"
query = "write a paragraph about the significance of TQM in the modern business world"
result = []
result = qa.run(query)

#append result  to the variable
book.append(section)
book.append(result)

#print the variable
print(book)

['#Definition of Total Quality Management (TQM)', 'Total Quality Management (TQM) is an integrated organizational effort designed to improve quality at every level of an organization. It is based on the principle that all departments within an organization work together to meet customer expectations and fulfill customer needs. TQM was first developed in Japan in the 1950s by Professor Kaoru Ishikawa. \n\nSince then, TQM has been adopted by organizations around the world to improve the quality of their products and services. TQM focuses on process improvement, customer satisfaction, and continual improvement. It uses tools such as quality control circles (QCCs), which were developed in the 1960s by Ishikawa, and total quality control (TQC), which was established in 1965 by the American Standards Association (ASA). \n\nTQM also uses techniques such as Quality Function Deployment (QFD), which was developed in the 1970s by the European Organization for Quality Control (EOQC), and the Europ

In [149]:
section = "#Introduction to Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)"
query = "write a paragraph about Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)"
result = []
book.append(section)
result = qa.run(query)
#append result  to the variable
book.append(result)

#print the variable
print(book)

['#Definition of Total Quality Management (TQM)', 'Total Quality Management (TQM) is an integrated organizational effort designed to improve quality at every level of an organization. It is based on the principle that all departments within an organization work together to meet customer expectations and fulfill customer needs. TQM was first developed in Japan in the 1950s by Professor Kaoru Ishikawa. \n\nSince then, TQM has been adopted by organizations around the world to improve the quality of their products and services. TQM focuses on process improvement, customer satisfaction, and continual improvement. It uses tools such as quality control circles (QCCs), which were developed in the 1960s by Ishikawa, and total quality control (TQC), which was established in 1965 by the American Standards Association (ASA). \n\nTQM also uses techniques such as Quality Function Deployment (QFD), which was developed in the 1970s by the European Organization for Quality Control (EOQC), and the Europ

In [150]:
section = "#Thesis Statement"
query = "write a Thesis statement about Evolution of Total Quality Management (TQM): The Contributions of Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)"
result = []
result = qa.run(query)

#append result  to the variable
book.append(section)
book.append(result)

#print the variable
print(book)

['#Definition of Total Quality Management (TQM)', 'Total Quality Management (TQM) is an integrated organizational effort designed to improve quality at every level of an organization. It is based on the principle that all departments within an organization work together to meet customer expectations and fulfill customer needs. TQM was first developed in Japan in the 1950s by Professor Kaoru Ishikawa. \n\nSince then, TQM has been adopted by organizations around the world to improve the quality of their products and services. TQM focuses on process improvement, customer satisfaction, and continual improvement. It uses tools such as quality control circles (QCCs), which were developed in the 1960s by Ishikawa, and total quality control (TQC), which was established in 1965 by the American Standards Association (ASA). \n\nTQM also uses techniques such as Quality Function Deployment (QFD), which was developed in the 1970s by the European Organization for Quality Control (EOQC), and the Europ

In [151]:
for entry in book:
#    print("Processing entry: ", entry)
    
    messages = [
        SystemMessage(content="You are an AI that takes python output data and translates it into well formatted markdown. Take {entry} and clean it up"),
        HumanMessage(content=entry)
    ]
    result = chat(messages)

    print(result)

content="Total Quality Management (TQM) is a management approach that seeks to optimize the quality of products and services by involving all employees in the organization in a continuous improvement process. TQM aims to meet or exceed customer expectations by ensuring that every aspect of the organization's operations is focused on quality. This includes processes, products, services, and the culture and values of the organization. TQM is based on the principle that quality is everyone's responsibility and that continuous improvement is essential to achieving success." additional_kwargs={} example=False
content="Total Quality Management (TQM) is an organizational approach that aims to improve the quality of products and services at every level of an organization. It is based on the principle of collaboration between all departments to meet customer expectations and needs. TQM was first developed in Japan in the 1950s by Professor Kaoru Ishikawa and has since been adopted by organizati

## II. Early History of Quality Management
- Brief description of quality management before TQM
- Introduction to foundational concepts: Inspection, Quality Control, and Quality Assurance
- The rise of statistical process control and its significance

In [ ]:
query = "write  Early History of Quality Management Brief description of quality management before TQM"
qa.run(query)
result = qa.run(query)

#append result  to the variable
book.append(result)

#print the variable
print(book)

['Definition of Total Quality Management (TQM)', 'Total Quality Management (TQM) is a systemic and data-driven approach to continuous improvement of organizational processes. TQM was first introduced in Japan in the 1950s by Professor Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE), and has since become a widely adopted quality management practice in the corporate world. \n\nTQM is characterized by its focus on customer satisfaction, its use of data-driven quality management tools and techniques, and its application of the principles of continuous improvement, including process mapping, root cause analysis, and control charts. Through these practices, TQM seeks to improve the quality of products and services, reduce costs, and increase customer satisfaction.', "Total Quality Management (TQM) is a business management strategy that has been used for decades to improve organizational performance and ensure customer satisfaction. It is an effective approach to incr

In [ ]:
query = "write Introduction to foundational concepts: Inspection, Quality Control, and Quality Assurance"
qa.run(query)
result = qa.run(query)

#append result  to the variable
book.append(result)

#print the variable
print(book)

['Definition of Total Quality Management (TQM)', 'Total Quality Management (TQM) is a systemic and data-driven approach to continuous improvement of organizational processes. TQM was first introduced in Japan in the 1950s by Professor Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE), and has since become a widely adopted quality management practice in the corporate world. \n\nTQM is characterized by its focus on customer satisfaction, its use of data-driven quality management tools and techniques, and its application of the principles of continuous improvement, including process mapping, root cause analysis, and control charts. Through these practices, TQM seeks to improve the quality of products and services, reduce costs, and increase customer satisfaction.', "Total Quality Management (TQM) is a business management strategy that has been used for decades to improve organizational performance and ensure customer satisfaction. It is an effective approach to incr

In [ ]:
query = "The rise of statistical process control and its significance"
qa.run(query)
result = qa.run(query)

#append result  to the variable
book.append(result)

#print the variable
print(book)

['Definition of Total Quality Management (TQM)', 'Total Quality Management (TQM) is a systemic and data-driven approach to continuous improvement of organizational processes. TQM was first introduced in Japan in the 1950s by Professor Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE), and has since become a widely adopted quality management practice in the corporate world. \n\nTQM is characterized by its focus on customer satisfaction, its use of data-driven quality management tools and techniques, and its application of the principles of continuous improvement, including process mapping, root cause analysis, and control charts. Through these practices, TQM seeks to improve the quality of products and services, reduce costs, and increase customer satisfaction.', "Total Quality Management (TQM) is a business management strategy that has been used for decades to improve organizational performance and ensure customer satisfaction. It is an effective approach to incr

## III. The Birth and Evolution of TQM
- Introduction to the concept of TQM
- Overview of the principles and practices of TQM
- The role of TQM in post-World War II Japan's industrial comeback
- Adoption and evolution of TQM in the United States and around the world

In [ ]:
query = "The rise of statistical process control and its significance"
qa.run(query)
result = qa.run(query)

#append result  to the variable
book.append(result)

#print the variable
print(book)

['Definition of Total Quality Management (TQM)', 'Total Quality Management (TQM) is a systemic and data-driven approach to continuous improvement of organizational processes. TQM was first introduced in Japan in the 1950s by Professor Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE), and has since become a widely adopted quality management practice in the corporate world. \n\nTQM is characterized by its focus on customer satisfaction, its use of data-driven quality management tools and techniques, and its application of the principles of continuous improvement, including process mapping, root cause analysis, and control charts. Through these practices, TQM seeks to improve the quality of products and services, reduce costs, and increase customer satisfaction.', "Total Quality Management (TQM) is a business management strategy that has been used for decades to improve organizational performance and ensure customer satisfaction. It is an effective approach to incr

In [ ]:
query = "The rise of statistical process control and its significance"
qa.run(query)
result = qa.run(query)

#append result  to the variable
book.append(result)

#print the variable
print(book)

['Definition of Total Quality Management (TQM)', 'Total Quality Management (TQM) is a systemic and data-driven approach to continuous improvement of organizational processes. TQM was first introduced in Japan in the 1950s by Professor Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE), and has since become a widely adopted quality management practice in the corporate world. \n\nTQM is characterized by its focus on customer satisfaction, its use of data-driven quality management tools and techniques, and its application of the principles of continuous improvement, including process mapping, root cause analysis, and control charts. Through these practices, TQM seeks to improve the quality of products and services, reduce costs, and increase customer satisfaction.', "Total Quality Management (TQM) is a business management strategy that has been used for decades to improve organizational performance and ensure customer satisfaction. It is an effective approach to incr

In [ ]:
query = "The rise of statistical process control and its significance"
qa.run(query)
result = qa.run(query)

#append result  to the variable
book.append(result)

#print the variable
print(book)

['Definition of Total Quality Management (TQM)', 'Total Quality Management (TQM) is a systemic and data-driven approach to continuous improvement of organizational processes. TQM was first introduced in Japan in the 1950s by Professor Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE), and has since become a widely adopted quality management practice in the corporate world. \n\nTQM is characterized by its focus on customer satisfaction, its use of data-driven quality management tools and techniques, and its application of the principles of continuous improvement, including process mapping, root cause analysis, and control charts. Through these practices, TQM seeks to improve the quality of products and services, reduce costs, and increase customer satisfaction.', "Total Quality Management (TQM) is a business management strategy that has been used for decades to improve organizational performance and ensure customer satisfaction. It is an effective approach to incr

## IV. The Role of Kaoru Ishikawa in the Development of TQM
- Introduction to Ishikawa: his background and career
- Ishikawa's contribution to the development of Quality Circles
- The development and importance of the Ishikawa (cause-and-effect) Diagram
- Ishikawa's philosophical contribution to TQM: the 'next process is your customer'


In [ ]:
query = "write about:The Role of Kaoru Ishikawa in the Development of TQM - Introduction to Ishikawa: his background and career"
qa.run(query)
result = qa.run(query)

#append result  to the variable
book.append(result)

#print the variable
print(book)

['Definition of Total Quality Management (TQM)', 'Total Quality Management (TQM) is a systemic and data-driven approach to continuous improvement of organizational processes. TQM was first introduced in Japan in the 1950s by Professor Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE), and has since become a widely adopted quality management practice in the corporate world. \n\nTQM is characterized by its focus on customer satisfaction, its use of data-driven quality management tools and techniques, and its application of the principles of continuous improvement, including process mapping, root cause analysis, and control charts. Through these practices, TQM seeks to improve the quality of products and services, reduce costs, and increase customer satisfaction.', "Total Quality Management (TQM) is a business management strategy that has been used for decades to improve organizational performance and ensure customer satisfaction. It is an effective approach to incr

In [ ]:
query = "write about:The Role of Kaoru Ishikawa in the Development of TQM - Ishikawa's contribution to the development of Quality Circles"
qa.run(query)
result = qa.run(query)

#append result  to the variable
book.append(result)

#print the variable
print(book)

['Definition of Total Quality Management (TQM)', 'Total Quality Management (TQM) is a systemic and data-driven approach to continuous improvement of organizational processes. TQM was first introduced in Japan in the 1950s by Professor Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE), and has since become a widely adopted quality management practice in the corporate world. \n\nTQM is characterized by its focus on customer satisfaction, its use of data-driven quality management tools and techniques, and its application of the principles of continuous improvement, including process mapping, root cause analysis, and control charts. Through these practices, TQM seeks to improve the quality of products and services, reduce costs, and increase customer satisfaction.', "Total Quality Management (TQM) is a business management strategy that has been used for decades to improve organizational performance and ensure customer satisfaction. It is an effective approach to incr

In [ ]:
query = "write about:The Role of Kaoru Ishikawa in the Development of TQM - The development and importance of the Ishikawa (cause-and-effect) Diagram"
qa.run(query)
result = qa.run(query)

#append result  to the variable
book.append(result)

#print the variable
print(book)

['Definition of Total Quality Management (TQM)', 'Total Quality Management (TQM) is a systemic and data-driven approach to continuous improvement of organizational processes. TQM was first introduced in Japan in the 1950s by Professor Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE), and has since become a widely adopted quality management practice in the corporate world. \n\nTQM is characterized by its focus on customer satisfaction, its use of data-driven quality management tools and techniques, and its application of the principles of continuous improvement, including process mapping, root cause analysis, and control charts. Through these practices, TQM seeks to improve the quality of products and services, reduce costs, and increase customer satisfaction.', "Total Quality Management (TQM) is a business management strategy that has been used for decades to improve organizational performance and ensure customer satisfaction. It is an effective approach to incr

In [ ]:
query = "write about:The Role of Kaoru Ishikawa in the Development of TQM - Ishikawa's philosophical contribution to TQM: the 'next process is your customer'"
qa.run(query)
result = qa.run(query)

#append result  to the variable
book.append(result)

#print the variable
print(book)

['Definition of Total Quality Management (TQM)', 'Total Quality Management (TQM) is a systemic and data-driven approach to continuous improvement of organizational processes. TQM was first introduced in Japan in the 1950s by Professor Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE), and has since become a widely adopted quality management practice in the corporate world. \n\nTQM is characterized by its focus on customer satisfaction, its use of data-driven quality management tools and techniques, and its application of the principles of continuous improvement, including process mapping, root cause analysis, and control charts. Through these practices, TQM seeks to improve the quality of products and services, reduce costs, and increase customer satisfaction.', "Total Quality Management (TQM) is a business management strategy that has been used for decades to improve organizational performance and ensure customer satisfaction. It is an effective approach to incr

## V. The Role of the Japanese Union of Scientists and Engineers (JUSE)
- Background of JUSE and its role in the development of quality management in Japan
- Introduction of the Deming Prize by JUSE and its significance
- JUSE's role in promoting TQM and its principles in Japan and globally


In [ ]:
query = "write about:The Role of Kaoru Ishikawa in the Development of TQM - Background of JUSE and its role in the development of quality management in Japan"
qa.run(query)
result = qa.run(query)

#append result  to the variable
book.append(result)

#print the variable
print(book)

['Definition of Total Quality Management (TQM)', 'Total Quality Management (TQM) is a systemic and data-driven approach to continuous improvement of organizational processes. TQM was first introduced in Japan in the 1950s by Professor Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE), and has since become a widely adopted quality management practice in the corporate world. \n\nTQM is characterized by its focus on customer satisfaction, its use of data-driven quality management tools and techniques, and its application of the principles of continuous improvement, including process mapping, root cause analysis, and control charts. Through these practices, TQM seeks to improve the quality of products and services, reduce costs, and increase customer satisfaction.', "Total Quality Management (TQM) is a business management strategy that has been used for decades to improve organizational performance and ensure customer satisfaction. It is an effective approach to incr

In [ ]:
query = "write about:The Role of Kaoru Ishikawa in the Development of TQM - ntroduction of the Deming Prize by JUSE and its significance"
qa.run(query)
#append result  to the variable
book.append(result)

#print the variable
print(book)

['Definition of Total Quality Management (TQM)', 'Total Quality Management (TQM) is a systemic and data-driven approach to continuous improvement of organizational processes. TQM was first introduced in Japan in the 1950s by Professor Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE), and has since become a widely adopted quality management practice in the corporate world. \n\nTQM is characterized by its focus on customer satisfaction, its use of data-driven quality management tools and techniques, and its application of the principles of continuous improvement, including process mapping, root cause analysis, and control charts. Through these practices, TQM seeks to improve the quality of products and services, reduce costs, and increase customer satisfaction.', "Total Quality Management (TQM) is a business management strategy that has been used for decades to improve organizational performance and ensure customer satisfaction. It is an effective approach to incr

In [ ]:
query = "write about:The Role of Kaoru Ishikawa in the Development of TQM - JUSE's role in promoting TQM and its principles in Japan and globally"
qa.run(query)
result = qa.run(query)

#append result  to the variable
book.append(result)

#print the variable
print(book)

['Definition of Total Quality Management (TQM)', 'Total Quality Management (TQM) is a systemic and data-driven approach to continuous improvement of organizational processes. TQM was first introduced in Japan in the 1950s by Professor Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE), and has since become a widely adopted quality management practice in the corporate world. \n\nTQM is characterized by its focus on customer satisfaction, its use of data-driven quality management tools and techniques, and its application of the principles of continuous improvement, including process mapping, root cause analysis, and control charts. Through these practices, TQM seeks to improve the quality of products and services, reduce costs, and increase customer satisfaction.', "Total Quality Management (TQM) is a business management strategy that has been used for decades to improve organizational performance and ensure customer satisfaction. It is an effective approach to incr

## VI. The Impact of TQM, Ishikawa, and JUSE on Contemporary Business Practices
- Analysis of the role TQM plays in today's business and management practices
- The legacy of Ishikawa's methods and tools in current quality management
- Influence of JUSE's principles in contemporary quality initiatives and awards


In [ ]:
query = "write about: The Impact of TQM, Ishikawa, and JUSE on Contemporary Business Practices - Analysis of the role TQM plays in today's business and management practices"
qa.run(query)
result = qa.run(query)

#append result  to the variable
book.append(result)

#print the variable
print(book)

['Definition of Total Quality Management (TQM)', 'Total Quality Management (TQM) is a systemic and data-driven approach to continuous improvement of organizational processes. TQM was first introduced in Japan in the 1950s by Professor Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE), and has since become a widely adopted quality management practice in the corporate world. \n\nTQM is characterized by its focus on customer satisfaction, its use of data-driven quality management tools and techniques, and its application of the principles of continuous improvement, including process mapping, root cause analysis, and control charts. Through these practices, TQM seeks to improve the quality of products and services, reduce costs, and increase customer satisfaction.', "Total Quality Management (TQM) is a business management strategy that has been used for decades to improve organizational performance and ensure customer satisfaction. It is an effective approach to incr

In [ ]:
query = "write about: The Impact of TQM, Ishikawa, and JUSE on Contemporary Business Practices - The legacy of Ishikawa's methods and tools in current quality management"
qa.run(query)
result = qa.run(query)

#append result  to the variable
book.append(result)

#print the variable
print(book)

['Definition of Total Quality Management (TQM)', 'Total Quality Management (TQM) is a systemic and data-driven approach to continuous improvement of organizational processes. TQM was first introduced in Japan in the 1950s by Professor Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE), and has since become a widely adopted quality management practice in the corporate world. \n\nTQM is characterized by its focus on customer satisfaction, its use of data-driven quality management tools and techniques, and its application of the principles of continuous improvement, including process mapping, root cause analysis, and control charts. Through these practices, TQM seeks to improve the quality of products and services, reduce costs, and increase customer satisfaction.', "Total Quality Management (TQM) is a business management strategy that has been used for decades to improve organizational performance and ensure customer satisfaction. It is an effective approach to incr

In [ ]:
query = "write about: The Impact of TQM, Ishikawa, and JUSE on Contemporary Business Practices - Influence of JUSE's principles in contemporary quality initiatives and awards"
qa.run(query)
result = qa.run(query)

#append result  to the variable
book.append(result)

#print the variable
print(book)

['Definition of Total Quality Management (TQM)', 'Total Quality Management (TQM) is a systemic and data-driven approach to continuous improvement of organizational processes. TQM was first introduced in Japan in the 1950s by Professor Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE), and has since become a widely adopted quality management practice in the corporate world. \n\nTQM is characterized by its focus on customer satisfaction, its use of data-driven quality management tools and techniques, and its application of the principles of continuous improvement, including process mapping, root cause analysis, and control charts. Through these practices, TQM seeks to improve the quality of products and services, reduce costs, and increase customer satisfaction.', "Total Quality Management (TQM) is a business management strategy that has been used for decades to improve organizational performance and ensure customer satisfaction. It is an effective approach to incr

## VII. Conclusion
- Summary of the importance and influence of TQM, Ishikawa, and JUSE in the field of quality management
- Restatement of Thesis
- Future implications of the principles and practices of TQM, Ishikawa, and JUSE in the global business landscape

In [ ]:
query = "write about: Summary of the importance and influence of TQM, Ishikawa, and JUSE in the field of quality management"
qa.run(query)
result = qa.run(query)

#append result  to the variable
book.append(result)

#print the variable
print(book)

['Definition of Total Quality Management (TQM)', 'Total Quality Management (TQM) is a systemic and data-driven approach to continuous improvement of organizational processes. TQM was first introduced in Japan in the 1950s by Professor Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE), and has since become a widely adopted quality management practice in the corporate world. \n\nTQM is characterized by its focus on customer satisfaction, its use of data-driven quality management tools and techniques, and its application of the principles of continuous improvement, including process mapping, root cause analysis, and control charts. Through these practices, TQM seeks to improve the quality of products and services, reduce costs, and increase customer satisfaction.', "Total Quality Management (TQM) is a business management strategy that has been used for decades to improve organizational performance and ensure customer satisfaction. It is an effective approach to incr

In [ ]:
query = "Restatement of Thesis"
qa.run(query)
result = qa.run(query)

#append result  to the variable
book.append(result)

#print the variable
print(book)

['Definition of Total Quality Management (TQM)', 'Total Quality Management (TQM) is a systemic and data-driven approach to continuous improvement of organizational processes. TQM was first introduced in Japan in the 1950s by Professor Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE), and has since become a widely adopted quality management practice in the corporate world. \n\nTQM is characterized by its focus on customer satisfaction, its use of data-driven quality management tools and techniques, and its application of the principles of continuous improvement, including process mapping, root cause analysis, and control charts. Through these practices, TQM seeks to improve the quality of products and services, reduce costs, and increase customer satisfaction.', "Total Quality Management (TQM) is a business management strategy that has been used for decades to improve organizational performance and ensure customer satisfaction. It is an effective approach to incr

In [ ]:
query = "write about: Future implications of the principles and practices of TQM, Ishikawa, and JUSE in the global business landscape"
qa.run(query)
result = qa.run(query)

#append result  to the variable
book.append(result)

#print the variable
print(book)

['Definition of Total Quality Management (TQM)', 'Total Quality Management (TQM) is a systemic and data-driven approach to continuous improvement of organizational processes. TQM was first introduced in Japan in the 1950s by Professor Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE), and has since become a widely adopted quality management practice in the corporate world. \n\nTQM is characterized by its focus on customer satisfaction, its use of data-driven quality management tools and techniques, and its application of the principles of continuous improvement, including process mapping, root cause analysis, and control charts. Through these practices, TQM seeks to improve the quality of products and services, reduce costs, and increase customer satisfaction.', "Total Quality Management (TQM) is a business management strategy that has been used for decades to improve organizational performance and ensure customer satisfaction. It is an effective approach to incr

In [ ]:
for entry in book:
#    print("Processing entry: ", entry)
    
    messages = [
        SystemMessage(content="You are an AI that takes dictionary data and translates it into well formatted markdown. You create sections and subsections to communicate as much information as possible"),
        HumanMessage(content=entry)
    ]
    result = chat(messages)

    print("Result: ", result)

Result:  content="# Total Quality Management (TQM)\n\nTotal Quality Management (TQM) is a management approach that seeks to optimize the quality of products and services by involving all employees in the organization in a continuous improvement process. TQM is a customer-focused approach that aims to meet or exceed customer expectations by improving all aspects of the organization's operations. It involves a commitment to quality by everyone in the organization, from top management to front-line employees.\n\n## Key Principles of TQM\n\nThe key principles of TQM include:\n\n1. Customer focus: TQM is focused on meeting or exceeding customer expectations.\n\n2. Continuous improvement: TQM involves a continuous improvement process that seeks to improve all aspects of the organization's operations.\n\n3. Employee involvement: TQM involves all employees in the organization in the improvement process.\n\n4. Process improvement: TQM focuses on improving processes to increase efficiency and ef